In [1]:
import pandas as pd
from spark_session import LocalSparkSession
from dataset import Dataset
from mr_id3 import MapReduceIDR3
from pyspark.mllib.tree import DecisionTree

In [2]:
from log import log
from sklearn.impute import SimpleImputer
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [3]:
%time
num_fields = [
    'age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
    'hours_per_week', ]

categorical_fields = [
    'workclass', 'education',
    'marital_status', 'occupation', 'relationship',
    'race', 'sex', 'native_country', ]

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


In [4]:
%time
target = 'label'
filename = '/Users/marcelovasconcellos/PycharmProjects/8INF919_Devoir1_Classification-distribuee-par-arbre-de-decision/dataset/adult.data'

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


In [5]:
%time
clusters = [2, 4, ]  # list(range(1, 2))
multiply = [5, ]  # list(range(1, 2))
metrics = []

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


In [6]:
spark = LocalSparkSession(2)
spark.start()

22/10/13 10:44:07 WARN Utils: Your hostname, Mac-Pro-de-MARCELO.local resolves to a loopback address: 127.0.0.1; using 192.168.0.62 instead (on interface en2)
22/10/13 10:44:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/13 10:44:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/13 10:44:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
dataset = Dataset(spark.spark, filename, num_fields, categorical_fields, target)
dataset.load()
dataset.one_hot_encode_categorical_fields()

2022-10-13 10:44:09,768 [INFO] Dataset : Starting
2022-10-13 10:44:09,770 [INFO] Dataset : Loading Dataset /Users/marcelovasconcellos/PycharmProjects/8INF919_Devoir1_Classification-distribuee-par-arbre-de-decision/dataset/adult.data


In [8]:
mr_id3 = MapReduceIDR3(dataset)
mr_id3.set_labeled_point()
mr_id3.split()
mr_id3.train()
mr_id3.predict()

2022-10-13 10:44:22,860 [INFO] MapReduceIDR3 : Starting


22/10/13 10:44:28 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 24 (TID 16): Attempting to kill Python Worker


/Users/marcelovasconcellos/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [9]:
1/0

ZeroDivisionError: division by zero

In [ ]:
dataset.df.columns

In [ ]:
1/0

In [ ]:
df = dataset.df

In [ ]:
df.show(2)

In [ ]:
from pyspark.mllib.linalg import Vectors
from pyspark.sql.types import Row

In [ ]:
df.rdd.map(lambda r: Row(
        label = r.label,
#         age = r.age,
#         fnlwgt = r.fnlwgt,
#         education_num = r.education_num,
#         capital_gain = r.capital_gain,
#         capital_loss = r.capital_loss,
#         hours_per_week = r.hours_per_week,
#         workclass_idx = r.workclass_idx,
#         education_idx = r.education_idx,
#         marital_status_idx = r.marital_status_idx,
#         occupation_idx = r.occupation_idx,
#         relationship_idx = r.relationship_idx,
#         race_idx = r.race_idx,
#         sex_idx = r.sex_idx,
#         native_country_idx = r.native_country_idx,
        features = Vectors.dense(
            r.age, 
            r.fnlwgt, 
            r.education_num, 
            r.capital_gain, 
            r.capital_loss, 
            r.hours_per_week, 
            r.workclass_idx, 
            r.education_idx, 
            r.marital_status_idx, 
            r.occupation_idx, 
            r.relationship_idx, 
            r.race_idx, 
            r.sex_idx, 
            r.native_country_idx
        ))
        ).toDF().show()

In [ ]:
df.rdd.map(lambda r: Row(
                    label = r.label,
                    age = r.age,
                    features = Vectors.dense(r.label, r.age))
                    ).toDF().show()

In [ ]:
df.withColumn("features", Vectors.dense([df.label, df.age])) \
  .show()

In [ ]:
dataset.string_indexer()


In [ ]:
dataset.assemble_features()

In [ ]:
dataset.df.show(5)

In [ ]:
dataset.df.toPandas().head(5)

In [ ]:
1/0

In [ ]:
mr_id3 = MapReduceIDR3(dataset)

In [ ]:
mr_id3.set_labeled_point()

In [ ]:
mr_id3.split()

In [ ]:
mr_id3.dataset.df.schema

In [ ]:
mr_id3.training_data.toDF()

In [ ]:
mr_id3.train()

In [ ]:
mr_id3.model = DecisionTree.trainClassifier(
            mr_id3.training_data,
            numClasses=2,
            categoricalFeaturesInfo={},
            impurity='entropy',
            maxDepth=5,
            maxBins=42,
        )

In [ ]:
mr_id3.predictions = mr_id3.model.predict(
            mr_id3.test_data.map(lambda x: x.features))

In [ ]:
labels_and_predictions = mr_id3.test_data.map(
            lambda lp: lp.label).zip(mr_id3.predictions)

In [ ]:
dfl = labels_and_predictions.toDF()
dfl.printSchema()
dfl.show()

In [ ]:
1/0 #labels_and_predictions

In [ ]:
mr_id3.errors = labels_and_predictions.filter(
            lambda lp: lp[0] != lp[1]).count() / float(mr_id3.test_data.count())

In [ ]:
metrics = BinaryClassificationMetrics(labels_and_predictions)

In [ ]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [ ]:
mr_id3.predict()

In [ ]:
mr_id3 = MapReduceIDR3(dataset)

In [ ]:
dataset.df.show(5)

In [ ]:
1/0

In [ ]:
%time
for n_clusters in clusters:
    for m_factor in multiply:
        spark = LocalSparkSession(n_clusters)
        spark.start()

        dataset = Dataset(spark.spark, filename, num_fields, categorical_fields, target)
        dataset.load()
        dataset.string_indexer()
        dataset.multiply_dataset(m_factor)

        mr_id3 = MapReduceIDR3(dataset)
        mr_id3.set_labeled_point()
        mr_id3.split()
        mr_id3.train()
        mr_id3.predict()
        
        metric = mr_id3.get_metrics()
        metric['length_rows'] = dataset.df.count()
        metric['m_factor'] = m_factor
        metric['n_clusters'] = n_clusters
        metrics.append(metric)

        spark.stop()

In [ ]:
%time
df = pd.DataFrame.from_dict(metrics)

In [ ]:
df